In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blocks = 16
batches = 32
max_iters = 30000
learning_rate = 1e-3
training_percent = 0.7
eval_iters = 500

cuda


In [2]:
with open('grammar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_len = len(chars)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '´', 'Á', 'à', 'á', 'æ', 'è', 'é', 'í', 'ò', 'ó', 'ù', 'ú', 'Ā', 'ā', 'Ă', 'ă', 'ē', 'ĕ', 'ħ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'ō', 'Ŏ', 'ŏ', 'œ', 'Ū', 'ū', 'ŭ', 'ȳ', '̄', '̆', 'Α', 'Κ', 'Λ', 'Ν', 'Ο', 'Τ', '‘', '’', '“', '”', '⪥', '\ufeff']


In [3]:
str2int = { ch:i for i,ch in enumerate(chars) }
int2str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [ str2int[c] for c in s ]
decode = lambda l: ''.join([ int2str[i] for i in l ])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57,   1,
         55,  78,   1,  26,  61,  71,  62,  72,   1,  26,  74,  71,  67,  68,
         76,   1,  54,  67,  57,   1,  73,  61,  58,   1,  38,  67,  65,  62,
         67,  58,   1,  27,  62,  72,  73,  71,  62,  55,  74,  73,  58,  57,
          0,  39,  71,  68,  68,  59,  71,  58,  54,  57,  62,  67,  60,   1,
         43,  58,  54,  66,   1,  54,  73,   1,  61,  73,  73,  69,  21,  10,
         10,  76,  76,  76,   9,  69,  60,  57,  69,   9,  67,  58,  73,   1,
          4,  43])


In [4]:
n = int(training_percent*len(data))
train = data[:n]
valid = data[n:]

def get_batch(split):
    data = train if split=='train' else valid
    ix = torch.randint(len(data) - blocks, (batches,))
    # print(ix)
    x = torch.stack([data[i:i+blocks] for i in ix])
    y = torch.stack([data[i+1:i+blocks+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[ 61,  62,  56,  61,   1,  73,  61,  58,   1,  65,  54,  72,  73,   0,
          54,  56],
        [  1,  54,  67,  57,   1,  76,  68,  74,  65,  57,   0,  61,  54,  75,
          58,   1],
        [ 73,  61,  62,  72,   1,  66,  54,  77,  62,  66,   1,  55,  58,   1,
          55,  68],
        [ 61,   1,  73,  61,  58,  78,   1,  73,  61,  74,  72,   1,  72,  74,
          56,  64],
        [  1, 122,  24,  67,   1,  28,  67,  73,  58,  71,  73,  54,  62,  67,
          62,  67],
        [ 71,  57,   1,  78,  68,  74,  67,  60,   1,  65,  54,  57,  62,  58,
          72,   1],
        [ 73,  52,   1,  59,  68,  71,  66,  58,  57,   1,  55,  78,   1,  54,
          57,  57],
        [ 58,   1,  69,  54,  72,  72,  62,  75,  58,   7,   1, 122,  32,   1,
          54,  66],
        [ 62,  67,  59,  62,  67,  62,  73,  62,  75,  58,   1,  66,  68,  68,
          57,  72],
        [  1,  38,  59,   0,  73,  61,  58,  72,  58,   1,  73,  61,  58,  71,
          58,   1],
  

In [5]:
x = train[:blocks]
y = train[1:blocks+1]
for t in range(blocks):
    ctx = x[:t+1]
    target = y[t]
    print('When input is', ctx, 'target is', target)

When input is tensor([125]) target is tensor(0)
When input is tensor([125,   0]) target is tensor(0)
When input is tensor([125,   0,   0]) target is tensor(0)
When input is tensor([125,   0,   0,   0]) target is tensor(0)
When input is tensor([125,   0,   0,   0,   0]) target is tensor(39)
When input is tensor([125,   0,   0,   0,   0,  39]) target is tensor(71)
When input is tensor([125,   0,   0,   0,   0,  39,  71]) target is tensor(68)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68]) target is tensor(57)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57]) target is tensor(74)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74]) target is tensor(56)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56]) target is tensor(58)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58]) target is tensor(57)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57]

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [7]:
class Bigram(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_len, vocab_len)
    def forward(self, index, targets=None):
        logits = self.token_embedding(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = Bigram(vocab_len)
m = model.to(device)

ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(m.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


]WĬbŭJ̄ZS*ăP̆Ig}6⪥u﻿y0”IΛŪ:”ĭ4/xīΛHmħ´oGXi?ΚJ1GÁΚypjgī3ch̆!ŭ﻿4ā7ΤuBSd49vīáFTĕ̄ŏ ztciĪĪ2Cc̆ ,V-WZ ̆ÁΟsŪùN[ŏ:J_jΟĕEL”]pĪ´_ó_ū﻿KY⪥5ĭ}īMnUoù?YBXB(ȳ/Dū-6eΤ5’”ĕ
qŏ)ĪGRe[Áz9orIó:ȳΑ.RkiGEĂqM)fX*bx0QVt”àCf_‘òŭpt
/qH42&flȳ,iSéBSxæ}èC5ùĂéhWr(ùú4Áœ/´n[sTa
ŭEj
b“m0TtΤJē̆ ;àZ!puΚ‘nĀPŏIΛW2zȳō }CpP1H]-W´W6CĪN.ēīpEiūEk;IōáDùE6F̆s0Ī]ħŪpúR!úó 5’0 íō!5M/2 hQsò9}??´WLĕéΚD_ùMΤDXēΛynp`zt̄̆&jwmħ80’úquhBù(E)O/i;à:gtēFi‘ōòG94´úaF´PI
ZO(X&ȳH̆w,WRúT-Wdwt,̆d!P?9lĕG-̆Ewħ´æxāvT’īi?èŏJNæ”ŏArīȳΝAŏ]mō´/4’ĪōĬMCDĬāH:œ⪥5ēZzr,f7nΟ


In [ ]:
%%time
optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, loss: {losses}')
    xb, yb = get_batch('train')
    logits, loss = m.forward(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

step: 0, loss: {'train': tensor(2.5306), 'val': tensor(2.6981)}
step: 500, loss: {'train': tensor(2.5328), 'val': tensor(2.6977)}
step: 1000, loss: {'train': tensor(2.5286), 'val': tensor(2.6958)}
step: 1500, loss: {'train': tensor(2.5328), 'val': tensor(2.6978)}
step: 2000, loss: {'train': tensor(2.5304), 'val': tensor(2.6923)}
step: 2500, loss: {'train': tensor(2.5346), 'val': tensor(2.6857)}
step: 3000, loss: {'train': tensor(2.5328), 'val': tensor(2.7069)}
step: 3500, loss: {'train': tensor(2.5294), 'val': tensor(2.7006)}
step: 4000, loss: {'train': tensor(2.5340), 'val': tensor(2.6925)}
step: 4500, loss: {'train': tensor(2.5272), 'val': tensor(2.6855)}
step: 5000, loss: {'train': tensor(2.5303), 'val': tensor(2.6939)}
step: 5500, loss: {'train': tensor(2.5288), 'val': tensor(2.6933)}
step: 6000, loss: {'train': tensor(2.5294), 'val': tensor(2.6960)}
step: 6500, loss: {'train': tensor(2.5310), 'val': tensor(2.6954)}
step: 7000, loss: {'train': tensor(2.5285), 'val': tensor(2.6993)}

In [9]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(m.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


_o thas.
wicun cloreathes
hod    nee.. sts it’s ofos, pr
Peno
thompan wirerd al “TERe “Ho  _FI.
Teeretis,
So adsive  eano
 higeve “Wes  oy_ze toff ly we “I Bud 3 peis ard ghine oul ike_stheaby _ he  ARAULEnd.
 br TIly hison  bube indiveak) t ior  Sy 1.”
Wh ay.
Fon iked  ath oveapler A o
gurel “Aprean  d c.”   don wa theris, wor qusie ans is, wungemiouorlles ouive

APy ale wiphinjurend f o s: omad d hote at ABefiothimivechise _wif  iumie mantindulind:
Veach;  nnd  wnorsuo t awitound vons, TISE tr
